pip install fastf1
pip install --upgrade pandas

In [1]:
import fastf1 as ff1
from fastf1 import plotting
from fastf1 import utils

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import numpy as np
import pandas as pd

In [2]:
# Enable the cache by providing the name of the cache folder
ff1.Cache.enable_cache('cache') 

In [3]:
year, grand_prix, session = 2022, 'Saudi Arabia', 'R'

In [4]:
race = ff1.get_session(year, grand_prix, session)
race.load() # This is new with Fastf1 v.2.2

core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v2.2.5]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
core        WARNING 	No lap data for driver 22
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 19 drivers: ['1', '16', '55', '11', '63', '31', '4', '10', '20', '44', '24', '27', '18', '23', '77', '14', '3', '6', '22']


In [5]:
racelaps = race.laps

In [6]:
racelaps['laptime2'] = racelaps['LapStartDate'].diff().shift(-1)

In [7]:
racelaps.loc[racelaps['laptime2'] <'-1 days +23:37:12.206000','laptime2']= racelaps['LapTime']

In [8]:
racelaps["acumulado"] = racelaps.groupby('Driver')['laptime2'].cumsum()

In [9]:
racelaps.loc[racelaps['acumulado'] <'-1 days +23:37:12.206000','acumulado']= racelaps['laptime2']

In [10]:
racelapsmin = racelaps.groupby('LapNumber')['acumulado'].min()
df_racelapsmin = pd.DataFrame(data=racelapsmin).reset_index().rename(columns={'acumulado':'min_acumulado'})
df_racelapsmin

,LapNumber,min_acumulado
0,1.0,0 days 00:01:36.291000
1,2.0,0 days 00:03:11.013000
2,3.0,0 days 00:04:45.609000
3,4.0,0 days 00:06:20.049000
4,5.0,0 days 00:07:54.545000
5,6.0,0 days 00:09:29.026000
6,7.0,0 days 00:11:03.847000
7,8.0,0 days 00:12:38.466000
8,9.0,0 days 00:14:13.291000
9,10.0,0 days 00:15:48.127000


In [11]:
racelaps = racelaps.merge(df_racelapsmin[["LapNumber",'min_acumulado']],how = 'left')

In [12]:
racelaps['Diff_primero']=racelaps['acumulado']-racelaps['min_acumulado']

In [13]:
conditionlist = [
    (racelaps['TrackStatus'] == '1') , (racelaps['TrackStatus'] == '264') , (racelaps['TrackStatus'] == '4'), (racelaps['TrackStatus'] == '2'), (racelaps['TrackStatus'] == '26'), (racelaps['TrackStatus'] == '6'), (racelaps['TrackStatus'] == '67'), (racelaps['TrackStatus'] == '7'), (racelaps['TrackStatus'] == '5') ]
choicelist = ['Verde','Virtual Safety Car','Safety Car','Amarilla','Safety Car','Virtual Safety Car','Virtual Safety Car','Verde','Roja']
racelaps['Bandera'] = np.select(conditionlist, choicelist, default='Otro')

In [14]:
racelaps['Pittime']= (racelaps.PitInTime - racelaps.PitOutTime.shift(-1))*-1

In [15]:
racelaps.loc[racelaps['Pittime'] <'-1 days +23:37:12.206000','Pittime']= np.nan

In [16]:
racelaps['Diferencia respecto al primero']=racelaps.Diff_primero.dt.total_seconds()

In [17]:
racelaps['Tiempo_pits']=racelaps.Pittime.dt.total_seconds()

In [18]:
racelaps['Tiempo_vuelta'] = racelaps['laptime2'].astype(str)

In [19]:
racelaps['Tiempo_vuelta']=racelaps['Tiempo_vuelta'].apply(lambda x: x.replace('0 days 00:',''))

In [20]:
racelaps['Tiempo_vuelta']=racelaps['Tiempo_vuelta'].apply(lambda x: x.rstrip('000'))

In [21]:
racelaps.head(55)

,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,...,LapStartDate,laptime2,acumulado,min_acumulado,Diff_primero,Bandera,Pittime,Diferencia respecto al primero,Tiempo_pits,Tiempo_vuelta
0,0 days 01:04:05.947000,1,NaT,1.0,1.0,0 days 00:21:33.199000,NaT,NaT,0 days 00:00:30.360000,0 days 00:00:29.775000,...,2022-03-27 17:03:27.573,0 days 00:01:38.384000,0 days 00:01:38.384000,0 days 00:01:36.291000,0 days 00:00:02.093000,Verde,NaT,2.093,NaN,01:38.384
1,0 days 01:05:41.222000,1,0 days 00:01:35.275000,2.0,1.0,NaT,NaT,0 days 00:00:35.387000,0 days 00:00:30.255000,0 days 00:00:29.633000,...,2022-03-27 17:05:05.957,0 days 00:01:35.275000,0 days 00:03:13.659000,0 days 00:03:11.013000,0 days 00:00:02.646000,Verde,NaT,2.646,NaN,01:35.275
2,0 days 01:07:16.112000,1,0 days 00:01:34.890000,3.0,1.0,NaT,NaT,0 days 00:00:35.145000,0 days 00:00:30.204000,0 days 00:00:29.541000,...,2022-03-27 17:06:41.232,0 days 00:01:34.890000,0 days 00:04:48.549000,0 days 00:04:45.609000,0 days 00:00:02.940000,Verde,NaT,2.940,NaN,01:34.89
3,0 days 01:08:50.812000,1,0 days 00:01:34.700000,4.0,1.0,NaT,NaT,0 days 00:00:35.025000,0 days 00:00:30.238000,0 days 00:00:29.437000,...,2022-03-27 17:08:16.122,0 days 00:01:34.700000,0 days 00:06:23.249000,0 days 00:06:20.049000,0 days 00:00:03.200000,Verde,NaT,3.200,NaN,01:34.7
4,0 days 01:10:25.621000,1,0 days 00:01:34.809000,5.0,1.0,NaT,NaT,0 days 00:00:35.107000,0 days 00:00:30.147000,0 days 00:00:29.555000,...,2022-03-27 17:09:50.822,0 days 00:01:34.809000,0 days 00:07:58.058000,0 days 00:07:54.545000,0 days 00:00:03.513000,Verde,NaT,3.513,NaN,01:34.809
5,0 days 01:12:00.461000,1,0 days 00:01:34.840000,6.0,1.0,NaT,NaT,0 days 00:00:35.157000,0 days 00:00:30.156000,0 days 00:00:29.527000,...,2022-03-27 17:11:25.631,0 days 00:01:34.840000,0 days 00:09:32.898000,0 days 00:09:29.026000,0 days 00:00:03.872000,Verde,NaT,3.872,NaN,01:34.84
6,0 days 01:13:35.095000,1,0 days 00:01:34.634000,7.0,1.0,NaT,NaT,0 days 00:00:35.083000,0 days 00:00:30.108000,0 days 00:00:29.443000,...,2022-03-27 17:13:00.471,0 days 00:01:34.634000,0 days 00:11:07.532000,0 days 00:11:03.847000,0 days 00:00:03.685000,Verde,NaT,3.685,NaN,01:34.634
7,0 days 01:15:09.916000,1,0 days 00:01:34.821000,8.0,1.0,NaT,NaT,0 days 00:00:35.108000,0 days 00:00:30.194000,0 days 00:00:29.519000,...,2022-03-27 17:14:35.105,0 days 00:01:34.821000,0 days 00:12:42.353000,0 days 00:12:38.466000,0 days 00:00:03.887000,Verde,NaT,3.887,NaN,01:34.821
8,0 days 01:16:44.748000,1,0 days 00:01:34.832000,9.0,1.0,NaT,NaT,0 days 00:00:35.150000,0 days 00:00:30.248000,0 days 00:00:29.434000,...,2022-03-27 17:16:09.926,0 days 00:01:34.832000,0 days 00:14:17.185000,0 days 00:14:13.291000,0 days 00:00:03.894000,Verde,NaT,3.894,NaN,01:34.832
9,0 days 01:18:20,1,0 days 00:01:35.252000,10.0,1.0,NaT,NaT,0 days 00:00:35.219000,0 days 00:00:30.413000,0 days 00:00:29.620000,...,2022-03-27 17:17:44.758,0 days 00:01:35.252000,0 days 00:15:52.437000,0 days 00:15:48.127000,0 days 00:00:04.310000,Verde,NaT,4.310,NaN,01:35.252


In [22]:
racelaps.to_csv('race_laps_Jedaah_2022.csv',index=False)

###  Pivoteos

laps_pivot = (pd.pivot_table(racelaps, values='laptime2', index=['LapNumber'],
                    columns=['Driver'], aggfunc=np.sum))

tablecum=(laps_pivot.cumsum(axis = 0))
tablecum

tablecum['laptime_cum_min'] = tablecum.min(axis = 1) 
tablecum

laps_pivot2 = (racelaps.pivot(
               values=['laptime2'],
               index=['LapNumber'],
               columns=['Driver'],
              ).reset_index())
laps_pivot2